## 打印可视化界面

In [1]:
from onekey_algo.datasets.image_loader import default_loader
from onekey_algo.custom.components.comp2 import show_cam_on_image
import torch
import os
import random
from onekey_algo import get_param_in_cwd
from onekey_algo.custom.components.comp2 import extract, init_from_model, init_from_onekey,target_layer_mapping
from onekey_algo.utils.MultiProcess import MultiProcess
import numpy as np
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import monai
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
matplotlib.use('Agg')

data_root = get_param_in_cwd('radio_dir')
samples = glob(os.path.join(r'D:\20240908-Vince\crop', '*'))
random.shuffle(samples)
def viz_sample(samples, thread_id):
    random.shuffle(samples)
    model_root = os.path.join(data_root, 'models', 'CV-0', 'resnet18')
    model, transformer, device = init_from_onekey(os.path.join(model_root, 'viz'))
    for n, m in model.named_modules():
        print('Feature name:', n, "|| Module:", m)
    target_layer = target_layer_mapping[os.path.basename(model_root) + '_2D']
#     return 
    gradcam = monai.visualize.GradCAM(nn_module=model, target_layers=target_layer)
#     return
    viz_dir = os.path.join(model_root, 'Grad-CAM')
    os.makedirs(viz_dir, exist_ok=True)
    for sample in samples[:int(len(samples) * 0.1)]:
        if os.path.exists(os.path.join(viz_dir, os.path.basename(sample))):
            continue
        img = default_loader(sample)
        sample_ = transformer(img)
        sample_  = sample_.view(1, *sample_.size()).to(device)
        res_cam = gradcam(x=sample_, class_idx=None)
        fig, axes = plt.subplots(1, 3, figsize=(12, 4), facecolor='white')
    #     axes[0].imshow(-res_cam[0][0].cpu(), cmap='jet')
        axes[0].imshow(img.resize(sample_.size()[2:]))
        axes[0].axis('off')
    #     plt.savefig(f"viz/{os.path.basename(sample).replace('.png', '_se.png')}", bbox_inches = 'tight')
    #     plt.show()
    #     plt.figure(figsize=(10, 10))
    #     plt.axis('off')
        imshow = axes[1].imshow(-res_cam[0][0].cpu(),cmap='jet')
        axes[1].axis('off')
        imshow = axes[2].imshow(show_cam_on_image(img.resize(sample_.size()[2:]), -res_cam[0][0].cpu(), use_rgb=True, reverse=False), 
                                cmap='jet')
        axes[2].axis('off')
        cax = fig.add_axes([0.92, 0.17, 0.02, axes[2].get_position().height]) 
        plt.colorbar(imshow, cax=cax)
        plt.savefig(f'{viz_dir}/{os.path.basename(sample).replace(".npy", ".png")}', bbox_inches = 'tight')
        plt.show()
        plt.close(fig)
viz_sample(samples, thread_id=1)
# MultiProcess(func=viz_sample, samples=samples, num_process=1).run()

[2024-09-15 20:42:10 - comp2.py: 164]	INFO	模型参数：{'pretrained': False, 'model_name': 'resnet18', 'num_classes': 2, 'in_channels': 3}


Feature name:  || Module: ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (re